In [ ]:
%cd /workspace


In [ ]:
import os
print("cwd:", os.getcwd())
print("dirs:", os.listdir())


In [ ]:
import tensorflow as tf, numpy as np, pandas as pd
from tensorflow.keras import layers, models, callbacks, mixed_precision

# ---------- data ----------
train_ds = tf.keras.utils.image_dataset_from_directory(
    "ships32", validation_split=0.15, subset="training",
    seed=42, label_mode="int", image_size=(32, 32), batch_size=256)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "ships32", validation_split=0.15, subset="validation",
    seed=42, label_mode="int", image_size=(32, 32), batch_size=256)

aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (aug(x) / 255., y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds   = val_ds.map(lambda x, y: (x / 255., y),     num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)


In [4]:


# ---------- model (<30 layers) ----------
mixed_precision.set_global_policy('mixed_float16')

def block(x, f):
    x = layers.Conv2D(f, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)              # ← changement ici
    return x

inp = layers.Input((32, 32, 3))
x = block(inp, 32);  x = block(x, 32);  x = layers.MaxPool2D()(x)
x = block(x, 64);   x = block(x, 64);   x = layers.MaxPool2D()(x)
x = block(x, 128);  x = block(x, 128);  x = layers.MaxPool2D()(x)
x = block(x, 256)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
out = layers.Dense(13, activation='softmax', dtype='float32')(x)

model = models.Model(inp, out)
print("Nombre de couches :", len(model.layers))  # 28

model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cbs = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.5, patience=4, verbose=1)
]


model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=cbs)




KeyboardInterrupt: 

In [1]:
# ---------- submission ----------
comp = np.load('ships_competition.npz')['X'] / 255.
pred = model.predict(comp, batch_size=512).argmax(1)
pd.DataFrame({'ID': np.arange(len(pred)), 'Category': pred}).to_csv('ships_competition.csv', index=False)


NameError: name 'np' is not defined

In [2]:
import os, numpy as np, pandas as pd
from PIL import Image
import torch, timm
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm.data.mixup import Mixup
from timm.scheduler import CosineLRScheduler
from timm.utils import NativeScaler

# 1) Dataset
class Ships32(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs, self.labels, self.tf = imgs, labels, transform
    def __len__(self): return len(self.imgs)
    def __getitem__(self, i):
        img = np.array(Image.open(self.imgs[i]))
        return self.tf(image=img)["image"], self.labels[i]

def make_datasets(root, split):
    ids, labs = [], []
    for cls,idx in {'coastguard':0,'containership':1,'corvette':2,'cruiser':3,
                    'cv':4,'destroyer':5,'ferry':6,'methanier':7,'sailing':8,
                    'smallfish':9,'submarine':10,'tug':11,'vsmallfish':12}.items():
        folder = os.path.join(root, cls)
        files  = sorted(os.listdir(folder))
        n      = len(files)
        cut1   = int(0.70*n); cut2=int(0.85*n)
        if split=="train": sel=files[:cut1]
        if split=="val":   sel=files[cut1:cut2]
        if split=="test":  sel=files[cut2:]
        for f in sel:
            ids.append(os.path.join(folder, f)); labs.append(idx)
    return ids, labs

train_tf = A.Compose([
    A.RandomResizedCrop(32,32,scale=(0.8,1.0)), A.Flip(), A.Rotate(15),
    A.OneOf([A.Cutout(num_holes=1, max_h_size=8, max_w_size=8),
             A.CoarseDropout(max_holes=1, max_height=8, max_width=8)], p=0.5),
    A.Normalize(), ToTensorV2()
])
val_tf = A.Compose([A.Resize(32,32), A.Normalize(), ToTensorV2()])

tr_ids, tr_lb = make_datasets("ships32", "train")
vl_ids, vl_lb = make_datasets("ships32", "val")

mixup = Mixup(mixup_alpha=0.2, cutmix_alpha=1.0, prob=0.5)
train_ds = Ships32(tr_ids, tr_lb, train_tf)
val_ds   = Ships32(vl_ids, vl_lb, val_tf)

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True,  num_workers=4, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

# 2) Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=13).to(device)

# 3) Optim, scheduler, scaler, SWA
opt   = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
steps = len(train_dl)*50  # ~50 epochs
sched = OneCycleLR(opt, max_lr=1e-2, total_steps=steps, pct_start=0.3)
swa_model = torch.optim.swa_utils.AveragedModel(model)
swa_start = len(train_dl)*30
scaler = NativeScaler()

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# 4) Training loop
best = 0
for epoch in range(1,51):
    model.train()
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        xb, yb = mixup(xb, yb)
        with autocast():
            logits = model(xb)
            loss   = criterion(logits, yb)
        opt.zero_grad()
        scaler(loss, opt)
        sched.step()
    # SWA update
    if epoch>=30: swa_model.update_parameters(model)

    # validation
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(1)
            correct += (preds==yb).sum().item()
            total   += yb.size(0)
    acc = correct/total
    if acc>best:
        best=acc
        torch.save(model.state_dict(),"best.pth")
    print(f"Epoch {epoch} ▶ val_acc {acc:.4f}")

# 5) SWA finalize & test
torch.optim.swa_utils.update_bn(train_dl, swa_model)
swa_model.to(device)
swa_model.eval()

comp = np.load('ships_competition.npz')['arr_0']
test_tf = A.Compose([A.Normalize(), ToTensorV2()])
preds = []
with torch.no_grad():
    for i in range(0, len(comp), 512):
        batch = np.stack([test_tf(image=x*255)["image"] for x in comp[i:i+512]])
        out   = swa_model(batch.to(device)).argmax(1).cpu().numpy()
        preds.append(out)
preds = np.hstack(preds)

pd.DataFrame({'ID':np.arange(len(preds)),'Category':preds})\
  .to_csv('ships_competition.csv', index=False)


ModuleNotFoundError: No module named 'timm'

In [3]:
cbs = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)
]
model.fit(train_gen, validation_data=val_gen, epochs=100, callbacks=cbs)


NameError: name 'callbacks' is not defined

In [ ]:
comp = np.load('ships_competition.npz')['arr_0'] / 255.
pred = model.predict(comp, batch_size=512).argmax(1)
pd.DataFrame({'ID': np.arange(len(pred)), 'Category': pred}).to_csv('ships_competition.csv', index=False)
